In [93]:
from sympy import symbols, sin, cos, pi, pprint, simplify, atan2, sqrt, atan, latex
from sympy.matrices import Matrix, Transpose
from IPython.display import display
import ipywidgets as widgets
from functools import reduce
import numpy as np
from importlib import reload
import kuka_arm.scripts.inverse_kinematics as ik

In [ ]:
import kuka_arm.scripts.inverse_kinematics as ik
reload(ik)

test_cases = {1:[[[2.16135,-1.42635,1.55109],
                  [0.708611,0.186356,-0.157931,0.661967]],
                  [1.89451,-1.44302,1.69366],
                  [-0.65,0.45,-0.36,0.95,0.79,0.49]],
              2:[[[-0.56754,0.93663,3.0038],
                  [0.62073, 0.48318,0.38759,0.480629]],
                  [-0.638,0.64198,2.9988],
                  [-0.79,-0.11,-2.33,1.94,1.14,-3.68]],
              3:[[[-1.3863,0.02074,0.90986],
                  [0.01735,-0.2179,0.9025,0.371016]],
                  [-1.1669,-0.17989,0.85137],
                  [-2.99,-0.12,0.94,4.06,1.29,-4.12]],
              4:[[[2.025334948912725, 0.9435231432919353, 1.3268344716975315], [-0.08137147213675607, -0.16717652116184525, 0.10456194492167763, 0.9769838760024222]],[1.74589694978219,0.873373350569398,1.23301330552001],[0,0,0,0,0,0]],
              5:[]}

# for thetas in ik.SolveIKLimits(*test_cases[4][0]):
#   pprint(thetas)
#   pprint(ik.SolveFK(thetas))

for thetas in ik.SolveIK(*test_cases[4][0], debug = False):
∏  pprint(ik.SolveFK(thetas))


In [125]:
%timeit ik.SolveIKCheapest([0]*6, *test_cases[3][0])

58.5 ms ± 7.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Latex Printout

In [ ]:
for name, matrix in zip('T0_1,T1_2,T2_3,T3_4,T4_5,T5_6,T6_EE'.split(','), ik.DHMatrices()):
  name = name[0] + '^' + name[1:]
  print(name + ' = ' + latex(matrix) + '\\\\')

In [99]:
ee_pos = Matrix(symbols('e_x,e_y,e_z'))
r_z, r_y, r_x = symbols('r_z, r_y, r_x')
R = ik.rot_z(r_z) * ik.rot_y(r_y) * ik.rot_x(r_x) * ik.R_gripper
T = ee_pos
print(latex(simplify(R)))
print(latex(Matrix(T)))

\left[\begin{matrix}\sin{\left (r_{x} \right )} \sin{\left (r_{z} \right )} + \sin{\left (r_{y} \right )} \cos{\left (r_{x} \right )} \cos{\left (r_{z} \right )} & - \sin{\left (r_{x} \right )} \sin{\left (r_{y} \right )} \cos{\left (r_{z} \right )} + \sin{\left (r_{z} \right )} \cos{\left (r_{x} \right )} & \cos{\left (r_{y} \right )} \cos{\left (r_{z} \right )}\\- \sin{\left (r_{x} \right )} \cos{\left (r_{z} \right )} + \sin{\left (r_{y} \right )} \sin{\left (r_{z} \right )} \cos{\left (r_{x} \right )} & - \sin{\left (r_{x} \right )} \sin{\left (r_{y} \right )} \sin{\left (r_{z} \right )} - \cos{\left (r_{x} \right )} \cos{\left (r_{z} \right )} & \sin{\left (r_{z} \right )} \cos{\left (r_{y} \right )}\\\cos{\left (r_{x} \right )} \cos{\left (r_{y} \right )} & - \sin{\left (r_{x} \right )} \cos{\left (r_{y} \right )} & - \sin{\left (r_{y} \right )}\end{matrix}\right]
\left[\begin{matrix}e_{x}\\e_{y}\\e_{z}\end{matrix}\right]


## Forward Kinematics Playground

In [45]:
T3_EE = simplify(T3_4*T4_5*T5_6*T6_EE)
R3_EE = T3_EE[0:3, 0:3]
print(latex(simplify(R3_EE)))
#pprint(simplify(sqrt(R3_EE[0,2]**2 + R3_EE[2, 2]**2)))

\left[\begin{matrix}- \sin{\left (q_{4} \right )} \sin{\left (q_{6} \right )} + \cos{\left (q_{4} \right )} \cos{\left (q_{5} \right )} \cos{\left (q_{6} \right )} & - \sin{\left (q_{4} \right )} \cos{\left (q_{6} \right )} - \sin{\left (q_{6} \right )} \cos{\left (q_{4} \right )} \cos{\left (q_{5} \right )} & - \sin{\left (q_{5} \right )} \cos{\left (q_{4} \right )}\\\sin{\left (q_{5} \right )} \cos{\left (q_{6} \right )} & - \sin{\left (q_{5} \right )} \sin{\left (q_{6} \right )} & \cos{\left (q_{5} \right )}\\- \sin{\left (q_{4} \right )} \cos{\left (q_{5} \right )} \cos{\left (q_{6} \right )} - \sin{\left (q_{6} \right )} \cos{\left (q_{4} \right )} & \sin{\left (q_{4} \right )} \sin{\left (q_{6} \right )} \cos{\left (q_{5} \right )} - \cos{\left (q_{4} \right )} \cos{\left (q_{6} \right )} & \sin{\left (q_{4} \right )} \sin{\left (q_{5} \right )}\end{matrix}\right]


In [184]:
T0_1,T1_2,T2_3,T3_4,T4_5,T5_6,T6_EE = ik.DHMatrices()
def e(q11, q22, q33, q44, q55, q66, joints):
  from kuka_arm.scripts.inverse_kinematics import q1,q2,q3,q4,q5,q6
  mat = reduce(lambda acc, M: acc * M, [T1_2,T2_3,T3_4,T4_5,T5_6,T6_EE][:(joints-1)], T0_1)
  return pprint(mat.evalf(subs={q1: q11, q2: q22, q3: q33, q4: q44, q5: q55, q6: q66}, prec = 5))

spec = (-3.14,3.14,0.01)
widgets.interact(e, q11=spec, q22=spec, q33=spec, q44=spec, q55=spec, q66=spec, joints=(2,7))

⎡-1.1796e-16  -5.8981e-17      1.0         1.85    ⎤
⎢                                                  ⎥
⎢5.8981e-17      -1.0      -5.8981e-17  -1.0911e-16⎥
⎢                                                  ⎥
⎢    1.0      5.8981e-17   1.1796e-16      1.946   ⎥
⎢                                                  ⎥
⎣     0            0            0           1.0    ⎦


<function __main__.e>

### IK error

In [166]:
data = np.loadtxt('data/queries_2018_02_06_12_48_09_439.csv', delimiter=',')

In [183]:
reload(ik)
errors = []
for row in data:
  ee_pos = np.array(row[0:3])
  quaternion = row[3:7]
  prev_thetas = row[7:13]
  thetas = ik.SolveIKCheapest(prev_thetas, ee_pos, quaternion)
  print(np.abs(prev_thetas - np.array(thetas).reshape((1, -1)))[0, 3:])
#   print(np.abs(prev_thetas - np.array(thetas).reshape(1, -1)))
  computed_pos = np.array(ik.SolveFK(thetas))
  computed_pos = computed_pos.reshape((1,-1))
  error = np.sqrt(np.double(np.sum(np.power(ee_pos - computed_pos, 2))))
  errors.append(error)

print('%.5f'%np.average(errors))

[5.69311939016356e-7 0.00331166651808493 4.49653055659255e-5]
[0.00734727528747969 0.0681212544755690 0.0111652637167001]
[0.00771062169755599 0.0681330282123308 0.0115230669871469]
[0.00771348313849485 0.0681495959650390 0.0115165480744319]
[0.00771457325986047 0.0681712102712771 0.0115043437540971]
[0.00872386445852591 0.0820324814837724 0.0129653940115335]
[0.00872512804032910 0.0820697529859466 0.0129381296301561]
[0.0106276007993227 0.102752450436032 0.0156962779623542]
[0.0106451297696570 0.102801834428822 0.0156488740009273]
[0.0144840784213738 0.144114972595036 0.0211274132976342]
[0.0145632073671003 0.144089238607604 0.0210238443404736]
[0.0125476215790470 0.125583878543757 0.0178983901784449]
[0.0126568825983950 0.125200403997570 0.0178371746650212]
[0.00452965073108613 0.0471569963061460 0.00635879773049242]
[0.00455040022770925 0.0467870969551323 0.00639398457902329]
[0.000334914281948429 0.0214398736229178 0.000887575557461329]
[0.000373330504439995 0.0190616843674587 0.00

In [ ]:
reload(ik)
row = data[0]
ee_pos = np.array(row[0:3])
quaternion = row[3:7]
prev_thetas = row[7:13]
for thetas in ik.SolveIKLimits(ee_pos, quaternion):
  print(np.abs(prev_thetas - np.array(thetas).reshape((1, -1)))[0, 3:])